<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
     Advanced Python
</h1>
<h2 style="color: white; position: absolute; top:36%; left:10%;">
    Iterators, Generators, Context Managers, and Decorators
</h2>


<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:58%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:63%; left:10%;">
    Data Scientist
</h3>
</div>

# The Iterator Protocol

Every Python programmer, even beginners, use iteratables frequently.  They just don't always think about the fact they are doing so.  Every time you write a loop, or a list comprehension, or the constructors `list()`, `set()`, or `tuple()`, you are using an iterable.

An important distinction has already arisen in this introduction.  Although often the two terms are used carelessly, *iterator* and *iterable* are two distinct kinds of things in Python.  However, a great many objects are both iterators and iterables, and hence often the distinction is easy to miss.

## Iterators

An *iterator* is entirely defined as an object that has a method named `.__next__()`; it **may**, on some call to that method, raise the exception `StopIteration`.  Once an iterator has *once* raised `StopIteration`, it is *exhausted*. According to the Python documentation:

> Once an iterator’s `.__next__()` method raises StopIteration, it must continue to do so on subsequent calls. Implementations that do not obey this property are deemed broken.

The built-in function `next()` is a shortcut for calling the `.__next__()` method of an object.

To demonstrate, let us look at a "non-broken" (but also almost useless) iterator.

In [1]:
class MyIterator:
    def __init__(self):
        self.n = 10
        self.exhausted = False
    def __next__(self):
        if self.exhausted or self.n == 13:
            self.exhausted = True
            raise StopIteration
        self.n += 1
        return self.n

In [2]:
my_iterator = MyIterator()
while True:
    print(next(my_iterator))

11
12
13


StopIteration: 

In [3]:
print(next(my_iterator))  # Continues to raise StopIteration

StopIteration: 

The problem is, that only being able to call `next()` on an object is not very flexible.  For example, being able to loop over an object to get each successive item would be a lot more convenient.

In [4]:
my_iterator = MyIterator()
for n in my_iterator:
    print(n)

TypeError: 'MyIterator' object is not iterable

## Iterables

In order to support a loop, or a list comprehension, or a constructor for a collection type, we instead need an **iterable**.  An iterable is simply an object that has a method named `.__iter__()` which return an *iterator* when called with no arguments.  The built-in function `iter()` is a shortcut for calling that method, although it is uncommon to call it explicitly. More often, `.__iter__()` gets called "behind" the scenes in a loop or the like.

In [5]:
class MyIterable:
    def __iter__(self):
        return MyIterator()

In [6]:
my_iterable = MyIterable()
iter(my_iterable)

That isn't yet so useful, but it suddenly becomes so when we put it in a loop and other contexts that want an iterable.

In [7]:
list(my_iterable)

[11, 12, 13]

In [8]:
[n/7 for n in my_iterable]

[1.5714285714285714, 1.7142857142857142, 1.8571428571428572]

In [9]:
for n in my_iterable:
    print(n)

11
12
13


In [10]:
for x in enumerate(my_iterable):
    print(x)

(0, 11)
(1, 12)
(2, 13)


In [11]:
list(zip(my_iterable, "abcdefghi"))

[(11, 'a'), (12, 'b'), (13, 'c')]

Notice that `my_iterable` is **not** an iterator.

In [12]:
next(my_iterable)

TypeError: 'MyIterable' object is not an iterator

## Dual-function objects

Many Python objects are both iterators and iterables.  For example, file objects do both things (and also a bunch more; they are also context managers, for example). We can open a Project Gutenberg dictionary of slang from 1913.  Or technically an excerpt from it, which I put in a file in the repository.

In [13]:
# Open a file, read some bytes from it.
slang = open('slang.txt')
print(slang.read(388), end='')

~A 1~, first-rate, the very best; “she’s a prime girl, she is; she is
A 1.”—_Sam Slick_. The highest classification of ships at Lloyd’s; common
term in the United States; also at Liverpool and other English seaports.
Another, even more intensitive form is “first-class, letter A, No. 1.”
Some people choose to say A I, for no reason, however, beyond that of
being different from others.



As an iterator, we might do this:

In [14]:
for i in range(5):
    print(next(slang), end='')

~Abigail~, a lady’s-maid; perhaps obtained from old comedies. Used in an
uncomplimentary sense. Some think the term is derived from Abigail Hill
(Mrs. Masham), lady-in-waiting to Queen Anne, and a typical ABIGAIL in
the way of intrigue.



As an iterable, we might use it a bit differently:

In [15]:
count = 1
for line in slang:
    print(line, end='')
    if (count := count+1) > 8:
        break

~About Right~, “to do the thing ABOUT RIGHT,” _i.e._, to do it
properly, soundly, correctly; “he guv it ’im ABOUT RIGHT,” _i.e._, he
beat him severely.

~Abraham-man~, a vagabond, such as were driven to beg about the country
after the dissolution of the monasteries.—_See_ BESS O’ BEDLAM,
_infra_. They are well described under the title of _Bedlam
Beggars_.—_Shakspeare’s K. Lear_, ii. 3.


---

So far this is rather formal, but in the next lesson we start to make this useful.